\title{Creating an IP Block for use in the PYNQ-Z1 using MyHDL}
\author{Steven K Armour}
\maketitle

# Referainces
Vivado 2015.2 CUSTOM IP PART I - Creating and Packaging Your IP Vivado
https://www.youtube.com/watch?v=BEQXV3eAZNs

Creating a Custom IP core using the IP Integrator
https://reference.digilentinc.com/learn/programmable-logic/tutorials/zybo-creating-custom-ip-cores/start

Johnson Counter
http://www.myhdl.org/docs/examples/jc2.html



# Introduction
This is a training project to demonstrate the use of MyHDL in making the core logic for new IP block. How to then Package the core logic into an IP using Xilinx's IP Packager. And finally how t then integrate all the work this far into an overlay for the PYNQ-Z1 BOard and writing a PYNQ Python Instruction set.

The So-Called spec. for this will be to implement a 4Bit Johnson Counter (Shift Regester) on the PYNQ-Z1 Board with outputs to the LEDs LD 3-0 (the leads found above the push buttons on the bottom left of the board). With shifting at ~60Hz. And with co Hardware and Software directional control. Where the Software control will be injected into the AXI Bus via PYNQ. And the Hardware Controls will be implemented via the switches SW1-0 on the PYNQ-Z1 (the push switches to the left of the afforded mentioned push buttons). Where the Software and Hardware controls are codominant via the latest version in the input of each interface against it previous state will then take control of the shift direction

In other words, if the switches are changed to an allowable state before a change to the AXI then the new input of the switches will then set the direction of the Johnson Counter. And *vice versa* such that if the switches are left to alone and a new (allowable) direction signal is sent to the AXI then the new AXI input will set the direction.

To implement this we will create three Verilog modules in myHDL and test them. Then there confirmed Verilog code will be synthesized to Verilog. Following that a new Viviado project will be started targeting the PYNQ-Z1 board will have the Verilog code added to. This will be followed by creating an IP block that will hold the Johnson counter and the AXI/Switch control code. From this, the final book design will be implemented and the overlay synthesized. Finally, the resulting overlay will be uploaded to the board and a Python PYNQ class will write to control the board via AXI alongside with the switch control

# Creating the HDL Code

the following assumes that you have some exposure to python and the jupyter notebook. And better yet myHDL. 

Import the necessary python modules.

In [1]:
from myhdl import *
import numpy as np
from myhdlpeek import Peeker
import random

## The first module
The first HDL module that will be developed is the one to decreases the 100MHz clock from the ZYNQ on the PYNQ-Z1 to a ~60Hz clock. This is needed to that the output of the upcoming Johnson counter will be human readable.

In order to slow the clock we will create a incremental register HDL Module `SlowClock` that for every clock cycle of the ZYNQ will increment a set length register by one till it 

In [6]:
#I have to figure out why the 13 bit reg works and not the 21 calculted
Base=100e6; Target=60

In [19]:
RegWidth=15

The following is the code for `SlowClock` 

In [20]:
def SlowClock(FastClk, SlowClk, RegWidthParm, rst):
    #I, O, P, O
    
    #overflow counter creation
    ClkCounter=Signal(modbv(0)[RegWidthParm:])
    
    #logic
    @always(FastClk.posedge, rst.negedge)
    def logic():
        #reset code
        if rst:
            ClkCounter.next=0
        
        else:
            # the overflow to triger the slow output
            if ClkCounter==2**RegWidthParm-1:
                SlowClk.next=1
            
            #non overflow to set clock output low
            else:
                SlowClk.next=0
                ClkCounter.next=ClkCounter+1
    
    return logic

Next, we test `SlowClock` via the following test bench verify it does what we intended it to do. Create a slow clock output signal from the fast clock signal from the ZYNQ.

In [21]:
Peeker.clear() 
FastClk=Signal(bool(0)); Peeker(FastClk, 'Fclk')
SlowClk=Signal(bool(0)); Peeker(SlowClk, 'Sclk')
rst=Signal(bool(0)); Peeker(rst, 'rst')

RegWidthParm=RegWidth

DUT=SlowClock(FastClk=FastClk, SlowClk=SlowClk, 
              RegWidthParm=RegWidthParm, rst=rst)

def SlowClock_TB():
    
    #need to figure this out to be the 100MHz clcok from the ZYNQ
    @always(delay(int(0.5*1e9/Base)))
    def clkGen():
            FastClk.next=not FastClk
            
            
    @instance
    def stimules():
        Store=0
        while 1:
            if SlowClk==1:
                #I need to write a better test bench then this
                print('Next')
            Store+=int(SlowClk)
            if Store==5:
                raise StopSimulation
            yield FastClk.posedge
            
    
    return instances()
sim = Simulation(DUT, SlowClock_TB(), *Peeker.instances()).run()


Next
Next
Next
Next
Next


Now that the behavior has been verified to work the myHDL code for `SlowClock` is synthesized to Verilog code `SlowClock.v` via the following

In [22]:
#need to figure out how to syth `RegWidthParm` as parmter at output
toVerilog(SlowClock, FastClk, SlowClk, RegWidthParm, rst)

## The Second model

The second model was not developed here but instead was taken from the examples on myHDL site (http://www.myhdl.org/docs/examples/jc2.html)

In [23]:
ACTIVE = 0
#create the states
DirType = enum('RIGHT', 'LEFT')

def jc2(goLeft, goRight, stop, clk, q):

    """ A bi-directional 4-bit Johnson counter with stop control.

    I/O pins:
    --------
    clk      : input free-running slow clock 
    goLeft   : input signal to shift left (active-low switch)
    goRight    : input signal to shift right (active-low switch)
    stop     : input signal to stop counting (active-low switch)
    q        : 4-bit counter output (active-low LEDs; q[0] is right-most)

    """
    
    #regesters
    dir = Signal(DirType.LEFT)
    run = Signal(False)
    
    #state machine
    @always(clk.posedge)
    def logic():
        
        #State machine logic
        # direction
        if goRight == ACTIVE:
            dir.next = DirType.RIGHT
            run.next = True
        elif goLeft == ACTIVE:
            dir.next = DirType.LEFT
            run.next = True
        # stop
        if stop == ACTIVE:
            run.next = False
        #State machine ouputs
        # counter action
        if run:
            if dir == DirType.LEFT:
                q.next[4:1] = q[3:]
                q.next[0] = not q[3]
            else:
                q.next[3:] = q[4:1]
                q.next[3] = not q[0]

    return logic

The testbench is a modified version of the testbench from the `SlowClock`'s test bench and the original testbench for the Johnson counter on the myHDL site. Where we use the same simulation for the ZYNQ clock via `clkGen` but then route that clock signal from there through an instance of `SlowClock` to then to `jc2`. This is done to verify the compatibility of `SlowClock` and `jc2`.

In [10]:
ACTIVE, INACTIVE = bool(0), bool(1)
Peeker.clear() 
FastClk=Signal(bool(0)); Peeker(FastClk, 'Fclk')
SlowClk=Signal(bool(0)); Peeker(SlowClk, 'Sclk')
rst=Signal(bool(0)); Peeker(rst, 'rst')
goLeft, goRight, stop = [Signal(INACTIVE) for i in range(3)]
q = Signal(intbv(0)[4:])

def test(jc2):

    

    @always(delay(int(0.5*1e9/Base)))
    def clkGen():
            FastClk.next=not FastClk
    
    SlowClockInt=SlowClock(FastClk=FastClk, SlowClk=SlowClk,
                           RegWidthParm=RegWidthParm,  rst=rst)

    jc2_inst = jc2(goLeft=goLeft, goRight=goRight, stop=stop, 
                   clk=SlowClk, q=q)

    @instance
    def stimulus():
        for i in range(3):
            yield FastClk.negedge
        for sig, nrcycles in ((goLeft, 10), (stop, 3), (goRight, 10)):
            sig.next = ACTIVE
            yield FastClk.negedge
            sig.next = INACTIVE
            for i in range(nrcycles-1):
                yield FastClk.negedge
        raise StopSimulation

    @instance
    def monitor():
        print ("goLeft goRight stop clk q")
        print ("-------------------------")
        while True:
            yield FastClk.negedge
            yield delay(1)
            print ("%d %d %d" % (goLeft, goRight, stop) ,)
            yield FastClk.posedge
            print ("C",)
            yield delay(1)
            print (bin(q, 4))

    return clkGen, SlowClockInt, jc2_inst, stimulus, monitor

In [11]:
sim = Simulation(test(jc2))
sim.run()

goLeft goRight stop clk q
-------------------------
1 1 1
C
0000
1 1 1
C
0000
0 1 1
C
0000
1 1 1
C
0000
1 1 1
C
0000
1 1 1
C
0000
1 1 1
C
0000
1 1 1
C
0000
1 1 1
C
0000
1 1 1
C
0000
1 1 1
C
0000
1 1 1
C
0000
1 1 0
C
0000
1 1 1
C
0000
1 1 1
C
0000
1 0 1
C
0000
1 1 1
C
0000
1 1 1
C
0000
1 1 1
C
0000
1 1 1
C
0000
1 1 1
C
0000
1 1 1
C
0000
1 1 1
C
0000
1 1 1
C
0000
1 1 1
C
0000


0

here we convert the myHDL `j2c` to `j2c.v`. 

In [12]:
toVerilog(jc2, goLeft, goRight, stop, SlowClk, q)

** ToVerilogWarning: Output port is read internally: q


## The third module
This model will take in both the AXI input from one the registers as well as the SWITCH inputs and from there will compere both inputs to there the last state of input and then the newest differentiating input on one of the two lines will then be used to set the Direction outputs that will end up going the Johnson counter `jc2`

In [13]:
def DirControl(AXIin, SWITCHin, goLeft_O, goRight_O, clk):
    AXILast=Signal(intbv(0)[len(AXIin):])
    SWITCHLast=Signal(intbv(0)[len(AXIin):])
    
    @always(clk.posedge)
    def logic():
        if AXIin!=AXILast:
            AXILast.next=AXIin
            
            if AXIin!=0 or AXIin!=3:
                goRight_O.next=AXIin[0]
                goLeft_O.next=AXIin[1]
        
        if SWITCHin!=SWITCHLast:
            SWITCHLast.next=SWITCHin
        
            if SWITCHin!=0 or SWITCHin!=3:
                goRight_O.next=SWITCHin[0]
                goLeft_O.next=SWITCHin[1]
                    
    
    return logic
            
    

The following testbench for `DirControl` is strictly for said module. 

In [15]:
Peeker.clear()

AXIin=Signal(intbv(0)[32:]); Peeker(AXIin, 'AXIin')
SWITCHin=Signal(intbv(0)[2:]); Peeker(SWITCHin, 'SWITCHin')
goLeft_O, goRight_O=[Signal(bool(0)) for _ in range(2)]; 
Peeker(goLeft_O, 'goLeft_O'); Peeker(goRight_O, 'goRight_O')
clk=Signal(bool(0)); Peeker(clk, 'clk')

DUT=DirControl(AXIin=AXIin, SWITCHin=SWITCHin, 
               goLeft_O=goLeft_O, goRight_O=goRight_O, clk=clk)

def DirControl_TB():
    
    @always(delay(1))
    def clkGen():
            clk.next=not clk
    
    @instance
    def stimulint():
        Count=0
        while 1:
            
            if Count==1:
                AXIin.next=0
            elif Count==2:
                SWITCHin.next=3
            elif Count==3:
                AXIin.next=1
            elif Count==4:
                SWITCHin.next=2
            elif Count==5:
                AXIin.next=2
            elif Count==6:
                SWITCHin.next=1
            elif Count==7:
                AXIin.next=3
            elif Count==8:
                SWITCHin.next=0
            
           
            elif Count%4==1:
                AXIin.next=random.randint(0, 3)
            

            elif Count%4==3:
                SWITCHin.next=random.randint(0, 3)
            elif Count==4*6:
                raise StopSimulation()
            
            
            
                
            
            Count+=1
            yield clk.posedge 
    
    return instances()


In [16]:
sim = Simulation(DUT, DirControl_TB(), *Peeker.instances()).run()
Peeker.to_wavedrom(tock=clk)

In [17]:
A=Peeker.to_html_table(tock=True)


Time,AXIin,SWITCHin,clk,goLeft_O,goRight_O
0,0,0,0,0,0
1,0,0,1,0,0
2,0,0,0,0,0
3,0,3,1,0,0
4,0,3,0,0,0
5,1,3,1,1,1
6,1,3,0,1,1
7,1,2,1,0,1
8,1,2,0,0,1
9,2,2,1,1,0


In [18]:
toVerilog(DirControl, AXIin, SWITCHin, 
               goLeft_O, goRight_O, clk)

In [24]:
# need to write a test bench that works with all three modules

# The Resulting IP and RTL 
The following is a UML Diagram that shows what we are ultimately going to try to wire up in Vivado from created modules `SlowClock`, `j2c`, `DirControl`.  Where `SlowClock` will be an RTL element and `j2c`, `DirControl`will be encapsulated into new IP block `j2c_ip` that will be created in Vivado via the *create and package IP* tool 

<img src="PYNQ_AXI_Learn_IPDiag.svg">